In [1]:
import pandas as pd


In [2]:
df15 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_15.csv")
df16 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_16.csv")
df17 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_17.csv")
df18 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_18.csv")
df19 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_19.csv")
df20 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_20.csv")
df21 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_21.csv")
df22 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_22.csv")
df23 = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\income\ny_income_23.csv")

In [3]:
df15['year'] = 2015
df16['year'] = 2016
df17['year'] = 2017
df18['year'] = 2018
df19['year'] = 2019
df20['year'] = 2020
df21['year'] = 2021
df22['year'] = 2022
df23['year'] = 2023

In [4]:
df_all = pd.concat([df15, df16, df17, df18, df19, df20, df21, df22, df23], ignore_index=True)


In [5]:
# Split NAME column into County and State
df_all[['County', 'State']] = df_all['NAME'].str.split(',', expand=True)

C:\Users\athar\AppData\Local\Temp\ipykernel_13004\61925457.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[['County', 'State']] = df_all['NAME'].str.split(',', expand=True)
C:\Users\athar\AppData\Local\Temp\ipykernel_13004\61925457.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[['County', 'State']] = df_all['NAME'].str.split(',', expand=True)


In [6]:
# Clean up extra spaces
df_all['County'] = df_all['County'].str.strip()
df_all['State'] = df_all['State'].str.strip()

In [7]:
# Drop the original NAME column if not needed
df_all = df_all.drop(columns=['NAME'])

In [8]:
# Delete row at index 0
df_all = df_all.drop(0)
# Reset index after deletion
df_all = df_all.reset_index(drop=True)

In [9]:
# Remove the word "County" from County column
df_all['County'] = df_all['County'].str.replace('County', '', regex=False).str.strip()

In [10]:
# Reorder columns: GEO_ID, year, County, State, then the rest
cols = df_all.columns.tolist()

In [11]:
# Ensure order
new_order = ['GEO_ID', 'year', 'County', 'State'] + [col for col in cols if col not in ['GEO_ID', 'year', 'County', 'State']]


In [12]:
df_all = df_all[new_order]

In [13]:
keep = [
    "GEO_ID", "NAME", "State", "County", "year",
    "S1901_C01_001E",      # total households
    "S1901_C01_012E"       # mean household income (avg per household)
]

In [14]:
# Filter to the columns that exist, then rename
existing = [c for c in keep if c in df_all.columns]

In [15]:
renamed = {
    "S1901_C01_001E": "Number of households",
    "S1901_C01_012E": "Average income (per household)",
    "NAME": "Geographic Area Name"
}

In [16]:
df_out = df_all[existing].rename(columns=renamed)

In [17]:
keep_county = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\appalachian_counties.csv")

In [18]:
keep_ny = keep_county[keep_county["state"] == "New York"]

In [19]:
ny_counties = set(keep_ny["county"])

In [20]:
# Keep only rows in df_out where 'County' is in that list
df_out6 = df_out[df_out["County"].isin(ny_counties)]

In [21]:
df_out6.head()

,GEO_ID,State,County,year,Number of households,Average income (per household)
1,0500000US36003,New York,Allegany,2015,18285,42776
3,0500000US36007,New York,Broome,2015,79132,46261
4,0500000US36009,New York,Cattaraugus,2015,31635,42601
6,0500000US36013,New York,Chautauqua,2015,53546,42993
7,0500000US36015,New York,Chemung,2015,35167,50320


In [22]:
df_out6.to_csv("df_out6.csv", index=False)